In [1]:
import wandb

from WindDataset import *

wandb.login()

wandb: Currently logged in as: bhavye-mathur. Use `wandb login --relogin` to force relogin


True

In [2]:
DEVICE = "mps"
QUANTILE = None

OUTPUT_SIZE = 1

LOSS_FUNC = torch.nn.MSELoss

In [8]:
WindDataset.init(1,  variable="U", quantile=QUANTILE,
                 normalised=True, geographical=True, cyclic_time=True, cyclic_coordinates=True,
                 combine_land=True, remove_lake=True, barometric=False,
                 predict_difference=False, mathur2022=False, device=DEVICE)

train = WindDataset("train")
validation = WindDataset("validation")
test = WindDataset("test")

del WindDataset.data

Loaded ../data/subset/U-NGCTCC-10000000.ft


KeyError: 'V'

In [4]:
INPUT_SIZE = train.x.shape[-1]
INPUT_SIZE

17

In [5]:
def get_dense_model(input_size: int,
                    hidden_sizes: list[int],
                    output_size: int,
                    activation_func: callable):
    layers = []

    for size in hidden_sizes:
        layers.append(torch.nn.Linear(input_size, size))
        layers.append(activation_func())
        input_size = size

    layers.append(torch.nn.Linear(input_size, output_size))

    return torch.nn.Sequential(*layers)

In [6]:
class SinusoidalLayer(torch.nn.Module):
    def __init__(self, size_in: int, size_out: int, device="mps", dtype=None):
        factory_kwargs = {'device': device, 'dtype': dtype}

        super().__init__()
        self.size_in = size_in
        self.size_out = size_out

        self.periods = torch.linspace(0.01, 100, size_out, **factory_kwargs).repeat((size_in, 1))

        self.weights = torch.nn.Parameter(torch.empty((size_out, size_out), **factory_kwargs))
        self.phase = torch.nn.Parameter(torch.zeros(size_out, **factory_kwargs))
        # self.bias = torch.nn.Parameter(torch.empty(size_out, **factory_kwargs))

        torch.nn.init.kaiming_uniform_(self.weights, a=2.236)
        fan_in, _ = torch.nn.init._calculate_fan_in_and_fan_out(self.weights)
        # bound = 1 / (fan_in ** 0.5)
        # torch.nn.init.uniform_(self.bias, -bound, bound)

    def forward(self, x):
        out = torch.mm(x, self.periods)
        out = torch.add(out, self.phase)
        out = torch.sin(out)
        out = torch.mm(out, self.weights)
        # out = torch.add(out, self.bias)
        return out


def get_sinusoidal_model(input_size: int,
                         periods: int,
                         hidden_sizes: list[int],
                         output_size: int,
                         activation_func: callable):
    layers = [SinusoidalLayer(input_size, periods),
              activation_func()]
    input_size = periods

    for size in hidden_sizes:
        layers.append(torch.nn.Linear(input_size, size))
        layers.append(activation_func())
        input_size = size

    layers.append(torch.nn.Linear(input_size, output_size))

    return torch.nn.Sequential(*layers)


In [7]:
def evaluate_one_epoch(model, epoch):
    mse = 0
    mae = 0

    with torch.no_grad():
        prediction = model(validation.x).squeeze()

        mse += torch.nn.functional.mse_loss(prediction, validation.y)
        mae += torch.nn.functional.l1_loss(prediction, validation.y)

    val_rmse = (mse ** 0.5) * stds[VARIABLE]

    wandb.log({"val_rmse": val_rmse,
               "val_mae": mae * stds[VARIABLE]})

    return val_rmse


def train_one_batch(model, optimizer, criterion, batch, batch_idx):
    optimizer.zero_grad()

    inputs, targets = batch

    prediction = model(inputs).squeeze()
    loss = criterion(prediction, targets)

    loss.backward()
    optimizer.step()

    if batch_idx != 0 and batch_idx % 100 == 0:
        with torch.no_grad():
            rmse = (torch.nn.functional.mse_loss(prediction, targets) ** 0.5) * stds[VARIABLE]
            wandb.log({"train_loss": loss,
                       "train_rmse": rmse})


def train_one_epoch(model, optimizer, criterion, epoch, batch_size):
    n = len(train)

    for i in range(len(train) // batch_size):
        lower_i = i * batch_size
        upper_i = min((i + 1) * batch_size, n)

        batch_x = train.x[lower_i: upper_i]
        batch_y = train.y[lower_i: upper_i]

        train_one_batch(model, optimizer, criterion, (batch_x, batch_y), i)


def main(conf=None):
    wandb.init(project=f"MERRA2-{VARIABLE}-July2023", config=conf)

    # learning_rate = wandb.config.learning_rate
    batch_size = wandb.config.batch_size
    layers = wandb.config.layers
    epochs = wandb.config.epochs
    activation = wandb.config.activation

    activation = getattr(torch.nn, activation)

    model = get_dense_model(INPUT_SIZE, layers, OUTPUT_SIZE, activation)
    model = model.to(DEVICE)
    print(model)

    criterion = LOSS_FUNC()
    optimizer = torch.optim.Adam(model.parameters())

    if (scheduler := wandb.config.lr_scheduler) is None:
        scheduler = None
    else:
        scheduler = getattr(torch.optim.lr_scheduler, scheduler)(optimizer, **wandb.config.lr_scheduler_kwargs)

    if DEVICE != "mps":
        wandb.watch(model, log_freq=10)

    for ep in tqdm(range(epochs)):
        print(ep, end=" ")

        wandb.log({"epoch": ep})

        model.train()
        train_one_epoch(model, optimizer, criterion, ep, batch_size)

        model.eval()
        val_rmse = evaluate_one_epoch(model, ep)

        if wandb.config.lr_scheduler == "ReduceLROnPlateau":
            scheduler.step(val_rmse)
            wandb.log({"lr": optimizer.param_groups[0]["lr"]})
        elif scheduler:
            scheduler.step()
            wandb.log({"lr": scheduler.get_last_lr()[-1]})

    # torch.save(model, os.path.join(wandb.run.dir, "model.h5"))
    return model

In [8]:
config = {
    "batch_size": 1024,
    "learning_rate": 0.001,
    # "lr_scheduler": "ReduceLROnPlateau",
    # "lr_scheduler_kwargs": {},

    "lr_scheduler": "StepLR",
    "lr_scheduler_kwargs": {"step_size": 18, "gamma": 0.7},

    # "lr_scheduler": None,

    "layers": [1024, 512, 256],
    "activation": "LeakyReLU",
    # "estimate_quantile": QUANTILE,
    "dataset": "NGCTCCCL",
    "epochs": 200,
}

model = main(config)
wandb.finish()

Sequential(
  (0): Linear(in_features=18, out_features=1024, bias=True)
  (1): LeakyReLU(negative_slope=0.01)
  (2): Linear(in_features=1024, out_features=512, bias=True)
  (3): LeakyReLU(negative_slope=0.01)
  (4): Linear(in_features=512, out_features=256, bias=True)
  (5): LeakyReLU(negative_slope=0.01)
  (6): Linear(in_features=256, out_features=1, bias=True)
)


  0%|          | 0/200 [00:00<?, ?it/s]

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 

wandb: Network error (ConnectionError), entering retry loop.


137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
lr,████▆▆▆▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_loss,█▇▇▆▆▆▆▃▅▃▄▂▄▄▄▃▅▄▃▄▂▃▂▃▄▂▃▂▃▁▃▂▂▃▁▃▂▃▁▃
train_rmse,█▇▇▆▆▆▆▃▆▃▄▂▄▅▄▃▅▄▃▄▂▄▂▃▄▂▃▂▃▁▃▂▃▄▁▃▂▃▁▄
val_mae,█▆▅▅▄▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_rmse,█▆▅▅▄▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,199
lr,2e-05
train_loss,0.23878
train_rmse,1.57932
val_mae,1.23565


In [10]:
torch.save(model, "FNN/V-1024-512-256-LeakyReLU-NGCTCC-radiant-wave-1")

In [ ]:
sweep_configuration = {
    "method": "bayes",
    "name": f"sweep-NGCTCC",
    "metric": {
        "goal": "minimize",
        "name": "val_rmse"
    },
    "parameters": {
        "batch_size": {"values": [1024, 2048, 4096, 8192]},
        # "learning_rate": {"max": 0.01, "min": 0.00001},
        "lr_scheduler": {"value": None},
        "lr_scheduler_kwargs": {"parameters": {"step_size": {"max": 20, "min": 10},
                                               "gamma": {"max": 0.85, "min": 0.5}}},
        "layers":
            # {"values": [(256, 128), (512, 256), (1024, 512), (2048, 1024)]},
            # {"values": [(256, 128), (512, 256), (1024, 512), (1024, 512, 256), (256, 256, 32), (512, 256, 128), (128, 64, 32)]},
            {"values": [(1024, 512, 256), (512, 256, 128), (256, 128, 64)]},
        "epochs": {"value": 100},
        "activation":
            {"values": ["ReLU", "PReLU", "LeakyReLU", "Tanh"]},
        # "estimate_quantile": {"value": QUANTILE},
        "dataset": {"value": "NGCTCC"},
    },
    "early_terminate": {
        "type": "hyperband",
        "min_iter": 3,
        "eta": 2
    }
}

sweep_id = wandb.sweep(sweep=sweep_configuration, project=f"MERRA2-{VARIABLE}-July2023")
wandb.agent(sweep_id, function=main)

In [43]:
model = torch.load("FNN/U-1024-512-256-LeakyReLU-NGCTCC-hardy-bee-540")
model.eval()

Sequential(
  (0): Linear(in_features=19, out_features=1024, bias=True)
  (1): LeakyReLU(negative_slope=0.01)
  (2): Linear(in_features=1024, out_features=512, bias=True)
  (3): LeakyReLU(negative_slope=0.01)
  (4): Linear(in_features=512, out_features=256, bias=True)
  (5): LeakyReLU(negative_slope=0.01)
  (6): Linear(in_features=256, out_features=1, bias=True)
)

In [11]:
with torch.no_grad():
    prediction = model(test.x).squeeze() * stds[VARIABLE] + means[VARIABLE]
    target = test.y * stds[VARIABLE] + means[VARIABLE]

    lines = f"""
    Original Stdev: {torch.std(target)} m/s
    Predicted MAE:  {torch.nn.functional.l1_loss(prediction, target)} m/s
    Predicted RMSE: {torch.nn.functional.mse_loss(prediction, target) ** 0.5} m/s
    """
    print(lines)


    Original Stdev: 3.2307777404785156 m/s
    Predicted MAE:  1.232912540435791 m/s
    Predicted RMSE: 1.7036985158920288 m/s
    
